In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import requests
import csv
import time
import imghdr
import re
import os
import pandas as pd

In [2]:
#,encoding="utf-8"
def save_2_file(name,items):
    with open(name, "a",encoding="utf-8") as f:
        for item in items:
            f.write("%s\n" % item)

In [3]:
# Configure Selenium options
options = Options()
#options.headless = True  # Run in headless mode, without opening a browser window
options.add_argument('-v')  # add the version flag
options.browser_version= '114.0.5735.110'  # set the version to 69.0
options.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"
options.add_argument(r"user-data-dir=C:\Users\kaveh\AppData\Local\Google\Chrome\User-Data\Default")

# Path to your ChromeDriver executable
driver_path = 'chromedriver.exe'

In [4]:
driver = webdriver.Chrome(service=Service(driver_path), options=options)

In [79]:
#get current window handle
p = driver.current_window_handle
#get first child window
chwd = driver.window_handles

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.112)
Stacktrace:
	GetHandleVerifier [0x00007FF7454352A2+57122]
	(No symbol) [0x00007FF7453AEA92]
	(No symbol) [0x00007FF74527E3AB]
	(No symbol) [0x00007FF74526ABE6]
	(No symbol) [0x00007FF74526B31E]
	(No symbol) [0x00007FF74526B250]
	(No symbol) [0x00007FF74525CC5A]
	(No symbol) [0x00007FF74525D85D]
	(No symbol) [0x00007FF7452E1717]
	(No symbol) [0x00007FF7452D6FA0]
	(No symbol) [0x00007FF7452AD2B1]
	(No symbol) [0x00007FF7452AE494]
	GetHandleVerifier [0x00007FF7456DEF82+2849794]
	GetHandleVerifier [0x00007FF745731D24+3189156]
	GetHandleVerifier [0x00007FF74572ACAF+3160367]
	GetHandleVerifier [0x00007FF7454C6D06+653702]
	(No symbol) [0x00007FF7453BA208]
	(No symbol) [0x00007FF7453B62C4]
	(No symbol) [0x00007FF7453B63F6]
	(No symbol) [0x00007FF7453A67A3]
	BaseThreadInitThunk [0x00007FFFF1767614+20]
	RtlU

In [6]:
chwd

['D2D5FFF0A34E0DA74CABA02A3D63985B']

In [30]:
driver.switch_to.window(chwd[0])

# Crawling posts

In [5]:
#//div[contains(@class,"css-1dbjc4n r-1jgb5lz r-1ye8kvj r-13qz1uu")]//img-->all images in post
#//div[contains(@data-testid,"cellInnerDiv")]//div[@class="css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu"]--->better for a list of only that persons posts
#the above with img that has atribute src!
#//div[contains(@class,"css-1dbjc4n r-16y2uox r-1wbh5a2 r-z5qs1h r-1777fci r-kzbkwu r-1e081e0 r-ttdzmv")]-->Under the pics

In [219]:
# Locate the parent element that contains the <li> tags
parent_element = driver.find_elements(By.XPATH,'//div[@class="css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu"]')

In [220]:
parent_element

[<selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_10995")>,
 <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_10996")>,
 <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_10997")>,
 <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_10998")>]

In [221]:
last_element = parent_element[-1]

In [222]:
last_element

<selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_10998")>

In [223]:
overal_img_urls = []
overal_alts = []
text_under_pic = []
full_text = []
posts = {}
index = 0

In [224]:
is_scroll_end = False
while True:
    for li in parent_element:
        alts = []
        img_urls = []
    #     post_text = []
        under_pic = []
        # Retrieve the image source
        try:
            image_src = li.find_elements(By.XPATH,'.//img')
            for img in image_src:
                img_urls.append(img.get_attribute("src"))
        except:
            img_urls.append("No_image")
        #image_src = li.find_elements(By.XPATH,'.//img[contains(@class,"ivm-view-attr__img--centered")]')
        for img in image_src:
            try:
                alts.append(img.get_attribute("alt"))
            except:
    #             try:
    #                 alt_image_src = li.find_element(By.XPATH,'.//div[contains(@class,"display-flex flex-column flex-grow-1 full-width")]').text
    #             except:
                alts.append("No_alt")
        try:
            post_text = li.text
        except:
            post_text = "No_text"
        try:
            under_pic = li.find_element(By.XPATH,'.//div[contains(@class,"css-1dbjc4n r-16y2uox r-1wbh5a2 r-z5qs1h r-1777fci r-kzbkwu r-1e081e0 r-ttdzmv")]').text
        except:
            under_pic = "No_under_text"
            #alt_image_src = li.find_element(By.XPATH,'.//div[contains(@class,"display-flex flex-column flex-grow-1 full-width")]').text
        #for elem in image_src:
        #img_urls.append(image_src.get_attribute("src"))
        overal_img_urls.extend(img_urls)
        #alts.append(alt_image_src.text)
        overal_alts.extend(alts)
        text_under_pic.append(under_pic)
        full_text.append(post_text)

    #     try:
    #         text_element = li.find_element(By.XPATH,'//div[contains(@class,"update-components-text relative feed-shared-update-v2__commentary")]')
    #         # Retrieve the text content
    #         tex = text_element.text
    #         overal_text.append(tex)
    #     except:
    #         pass
        posts[f"Post{index}"] = [img_urls,alts,post_text,under_pic]

        index+=1
        # Do something with the image source and text
        print("Post number:", index)

    driver.execute_script("arguments[0].scrollIntoView();", last_element)
    time.sleep(1)
    parent_element = driver.find_elements(By.XPATH,'//div[@class="css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu"]')
    print("length of new scraping list: ",len(parent_element))
    # Check if the scroll has reached the end
    scroll_height = driver.execute_script("return document.documentElement.scrollHeight")
    inner_height = driver.execute_script("return window.innerHeight")
    is_scroll_end = scroll_height == inner_height

    if is_scroll_end:
        print("Reached the end of the page.")
        break
    last_element = parent_element[-1]
    print("the last element: ",last_element)
    time.sleep(5)
    

Post number: 1
Post number: 2
Post number: 3
Post number: 4
length of new scraping list:  8
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_11013")>
Post number: 5
Post number: 6
Post number: 7
Post number: 8
Post number: 9
Post number: 10
Post number: 11
Post number: 12
length of new scraping list:  13
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_11031")>
Post number: 13
Post number: 14
Post number: 15
Post number: 16
Post number: 17
Post number: 18
Post number: 19
Post number: 20
Post number: 21
Post number: 22
Post number: 23
Post number: 24
Post number: 25
length of new scraping list:  11
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_1104

Post number: 201
Post number: 202
Post number: 203
Post number: 204
Post number: 205
Post number: 206
Post number: 207
length of new scraping list:  7
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_11388")>
Post number: 208
Post number: 209
Post number: 210
Post number: 211
Post number: 212
Post number: 213
Post number: 214
length of new scraping list:  11
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_11412")>
Post number: 215
Post number: 216
Post number: 217
Post number: 218
Post number: 219
Post number: 220
Post number: 221
Post number: 222
Post number: 223
Post number: 224
Post number: 225
length of new scraping list:  7
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3D

Post number: 410
Post number: 411
Post number: 412
Post number: 413
Post number: 414
Post number: 415
Post number: 416
Post number: 417
Post number: 418
Post number: 419
length of new scraping list:  10
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_11826")>
Post number: 420
Post number: 421
Post number: 422
Post number: 423
Post number: 424
Post number: 425
Post number: 426
Post number: 427
Post number: 428
Post number: 429
length of new scraping list:  6
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_11841")>
Post number: 430
Post number: 431
Post number: 432
Post number: 433
Post number: 434
Post number: 435
length of new scraping list:  4
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", elem

length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_12267")>
Post number: 616
Post number: 617
Post number: 618
Post number: 619
Post number: 620
length of new scraping list:  8
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_12277")>
Post number: 621
Post number: 622
Post number: 623
Post number: 624
Post number: 625
Post number: 626
Post number: 627
Post number: 628
length of new scraping list:  8
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_12297")>
Post number: 629
Post number: 630
Post number: 631
Post number: 632
Post number: 633
Post number: 634
Post number: 635
Post number: 636
length of new scraping list:  8

Post number: 811
Post number: 812
Post number: 813
Post number: 814
length of new scraping list:  11
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_12641")>
Post number: 815
Post number: 816
Post number: 817
Post number: 818
Post number: 819
Post number: 820
Post number: 821
Post number: 822
Post number: 823
Post number: 824
Post number: 825
length of new scraping list:  8
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_12665")>
Post number: 826
Post number: 827
Post number: 828
Post number: 829
Post number: 830
Post number: 831
Post number: 832
Post number: 833
length of new scraping list:  7
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_12676

...

Post number: 1944
Post number: 1945
Post number: 1946
Post number: 1947
Post number: 1948
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 1949
Post number: 1950
Post number: 1951
Post number: 1952
Post number: 1953
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 1954
Post number: 1955
Post number: 1956
Post number: 1957
Post number: 1958
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 1959
Post number: 1960
Post number: 1961
Post number: 1962
Post number: 1963
length of new scraping list:

Post number: 2084
Post number: 2085
Post number: 2086
Post number: 2087
Post number: 2088
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 2089
Post number: 2090
Post number: 2091
Post number: 2092
Post number: 2093
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 2094
Post number: 2095
Post number: 2096
Post number: 2097
Post number: 2098
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 2099
Post number: 2100
Post number: 2101
Post number: 2102
Post number: 2103
length of new scraping list:

Post number: 2224
Post number: 2225
Post number: 2226
Post number: 2227
Post number: 2228
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 2229
Post number: 2230
Post number: 2231
Post number: 2232
Post number: 2233
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 2234
Post number: 2235
Post number: 2236
Post number: 2237
Post number: 2238
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 2239
Post number: 2240
Post number: 2241
Post number: 2242
Post number: 2243
length of new scraping list:

Post number: 2364
Post number: 2365
Post number: 2366
Post number: 2367
Post number: 2368
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 2369
Post number: 2370
Post number: 2371
Post number: 2372
Post number: 2373
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 2374
Post number: 2375
Post number: 2376
Post number: 2377
Post number: 2378
length of new scraping list:  5
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14392")>
Post number: 2379
Post number: 2380
Post number: 2381
Post number: 2382
Post number: 2383
length of new scraping list:

KeyboardInterrupt: 

In [225]:
posts

{'Post0': [['https://pbs.twimg.com/card_img/1698739404561338368/WYoITIXP?format=webp&name=small'],
  ['Defence Secretary in Olsztyn, Poland.'],
  'Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nOct 1, 2022\nAn article by the Defence Secretary on the situation in Ukraine - http://GOV.UK\ngov.uk\nAn article by the Defence Secretary on the situation in Ukraine\nSecretary of State for Defence Ben Wallace discusses NATO, Ukraine and Russia.\n2,709\n574\n2,251',
  'gov.uk\nAn article by the Defence Secretary on the situation in Ukraine\nSecretary of State for Defence Ben Wallace discusses NATO, Ukraine and Russia.'],
 'Post1': [['https://abs-0.twimg.com/emoji/v2/svg/1f1e9-1f1f0.svg',
   'https://abs-0.twimg.com/emoji/v2/svg/1f4aa.svg',
   'https://abs-0.twimg.com/emoji/v2/svg/1f1fa-1f1e6.svg',
   'https://pbs.twimg.com/media/F5VyHLyW0AA-v94?format=webp&name=small'],
  ['🇩🇰', '💪', '🇺🇦', 'Image'],
  'Morten Bødskov\n@mfMorten\n·\nSep 6\nThank you, my friend \nThe last day for my good friend, \n@oleks

In [226]:
df = pd.DataFrame.from_dict(posts).T.astype(str).drop_duplicates().T

In [227]:
df

,Post0,Post1,Post2,Post3,Post8,Post9,Post10,Post11,Post16,Post17,...,Post1837,Post1842,Post1847,Post1852,Post1857,Post1862,Post1867,Post1872,Post1877,Post1882
0,['https://pbs.twimg.com/card_img/1698739404561...,['https://abs-0.twimg.com/emoji/v2/svg/1f1e9-1...,[],['https://pbs.twimg.com/media/F42c1TJXEAAHkOX?...,['https://abs-0.twimg.com/emoji/v2/svg/1f1eb-1...,[],[],[],[],['https://pbs.twimg.com/card_img/1697343487921...,...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...
1,"['Defence Secretary in Olsztyn, Poland.']","['🇩🇰', '💪', '🇺🇦', 'Image']",[],['Image'],"['🇫🇷', '🇬🇧', 'Image']",[],[],[],[],[''],...,"['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']"
2,"Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nOct 1,...",Morten Bødskov\n@mfMorten\n·\nSep 6\nThank you...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nSep 5\...,Oleksii Reznikov\n@oleksiireznikov\n·\nAug 31\...,Sébastien Lecornu\n@SebLecornu\n·\nAug 31\nÀ \...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nAug 31...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nJul 16...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nJul 16...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nAug 30...,"Mick Ryan, AM\n@WarintheFuture\n·\nAug 24\n“Wa...",...,"Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20..."
3,gov.uk\nAn article by the Defence Secretary on...,No_under_text,No_under_text,No_under_text,No_under_text,gov.uk\nBen Wallace's resignation letter and t...,No_under_text,No_under_text,No_under_text,washingtonpost.com\nOpinion | Ukraine’s counte...,...,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text


In [228]:
df.iloc[2,2]

'Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nSep 5\nAnd this is how it ends Mr Putin…the once mighty Russia scrabbling around looking for friends and begging North Korea for weapons from the 1960’s.\n825\n1,792\n10.9K\n726.9K'

In [229]:
save_2_file("overal_img_urls.txt",overal_img_urls)
save_2_file("overal_alts.txt",overal_alts)
save_2_file("text_under_pic.txt",text_under_pic)
save_2_file("full_text.txt",full_text)

In [230]:
df.to_csv("All_Posts.csv")

In [231]:
df_r = pd.read_csv("All_Posts.csv")

In [232]:
df_r

,Unnamed: 0,Post0,Post1,Post2,Post3,Post8,Post9,Post10,Post11,Post16,...,Post1837,Post1842,Post1847,Post1852,Post1857,Post1862,Post1867,Post1872,Post1877,Post1882
0,0,['https://pbs.twimg.com/card_img/1698739404561...,['https://abs-0.twimg.com/emoji/v2/svg/1f1e9-1...,[],['https://pbs.twimg.com/media/F42c1TJXEAAHkOX?...,['https://abs-0.twimg.com/emoji/v2/svg/1f1eb-1...,[],[],[],[],...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...,['https://abs-0.twimg.com/emoji/v2/svg/1f1ec-1...
1,1,"['Defence Secretary in Olsztyn, Poland.']","['🇩🇰', '💪', '🇺🇦', 'Image']",[],['Image'],"['🇫🇷', '🇬🇧', 'Image']",[],[],[],[],...,"['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']","['🇬🇧', '🎅']"
2,2,"Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nOct 1,...",Morten Bødskov\n@mfMorten\n·\nSep 6\nThank you...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nSep 5\...,Oleksii Reznikov\n@oleksiireznikov\n·\nAug 31\...,Sébastien Lecornu\n@SebLecornu\n·\nAug 31\nÀ \...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nAug 31...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nJul 16...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nJul 16...,Rt. Hon Ben Wallace MP\n@BWallaceMP\n·\nAug 30...,...,"Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20...","Ministry of Defence\n@DefenceHQ\n·\nDec 18, 20..."
3,3,gov.uk\nAn article by the Defence Secretary on...,No_under_text,No_under_text,No_under_text,No_under_text,gov.uk\nBen Wallace's resignation letter and t...,No_under_text,No_under_text,No_under_text,...,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text,No_under_text


In [233]:
df_r.iloc[0,3][2:-2].replace("'","").replace(" ","").split(",")

['']

In [234]:
df_r.columns[1]

'Post0'

In [235]:
for i in range(1,len(df_r.columns)):
    j = 0
    try:
        for url in df_r.iloc[0,i][2:-2].replace("'","").replace(" ","").split(","):
            try:
                response = requests.get(url)
                file = open(f"{df_r.columns[i]}_number{j}.jpeg", "wb")
                file.write(response.content)
                file.close()
            except:
                pass
            j+=1
#         file = open(f"Post{i}.jpeg", "wb")
#         file.write(response.content)
#         file.close()
    except Exception as e:
        print(f'An error occurred: {str(e)}')
        time.sleep(1)
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# Scraping Followers AND Following(Same code for both)

In [6]:
#//div[@data-testid="cellInnerDiv"]-->for each cell
# //div[@data-testid="cellInnerDiv"]//div[@class="css-1dbjc4n r-1awozwy r-18u37iz r-dnmrzs"]--for name of profiles
# //div[@data-testid="cellInnerDiv"]//img[@class="css-9pa8cd"]-->profile pictures

In [254]:
# Locate the parent element that contains the <li> tags
parent_element = driver.find_elements(By.XPATH,'//div[@data-testid="cellInnerDiv"]')

In [255]:
parent_element

[<selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14808")>,
 <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14809")>,
 <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14810")>,
 <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14811")>,
 <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14812")>,
 <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14813")>,
 <selenium.webdriver.remote.webelement.WebElement (session="32ba117fac

In [256]:
last_element = parent_element[-1]

In [257]:
last_element

<selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_14828")>

In [258]:
overal_img_urls = []
overal_alts = []
full_text = []
profiles = {}
index = 0

In [262]:
for li in parent_element:
#     post_text = []
    under_pic = []
    # Retrieve the image source
    try:
        img_urls = li.find_element(By.XPATH,'.//img[@class="css-9pa8cd"]')
        img_urls = img_urls.get_attribute("src")
    except:
        img_urls = "No_image"
    #image_src = li.find_elements(By.XPATH,'.//img[contains(@class,"ivm-view-attr__img--centered")]')
    try:
        alts = li.find_element(By.XPATH,'.//div[@class="css-1dbjc4n r-1awozwy r-18u37iz r-dnmrzs"]').text
    except:
        alts = f"No_alt{index}"
    try:
        profile_text = li.text
    except:
        profile_text = "No_text"
    overal_img_urls.append(img_urls)
    overal_alts.append(alts)
    full_text.append(profile_text)
    profiles[f"{alts}"] = [img_urls,alts,profile_text]
    
    index+=1
    # Do something with the image source and text
    print("Profile number:", index)

driver.execute_script("arguments[0].scrollIntoView();", last_element)
time.sleep(1)
parent_element = driver.find_elements(By.XPATH,'//div[@data-testid="cellInnerDiv"]')
print("lenght of new scraping list: ",len(parent_element))
last_element = parent_element[-1]
print("the last element: ",last_element)

Profile number: 104
Profile number: 105
Profile number: 106
Profile number: 107
Profile number: 108
Profile number: 109
Profile number: 110
Profile number: 111
Profile number: 112
Profile number: 113
Profile number: 114
Profile number: 115
Profile number: 116
Profile number: 117
Profile number: 118
Profile number: 119
Profile number: 120
Profile number: 121
Profile number: 122
lenght of new scraping list:  19
the last element:  <selenium.webdriver.remote.webelement.WebElement (session="32ba117facbef2028069cc4ec1d7f2e4", element="BE5DE3610C3DC6BA5B3435395D6483DE_element_15007")>


In [263]:
profiles

{'James Cleverly': ['https://pbs.twimg.com/profile_images/1639203057094340611/itDFS722_bigger.jpg',
  'James Cleverly',
  'James Cleverly\n@JamesCleverly\nFollow\nForeign Secretary of the United Kingdom. MP for Braintree.'],
 'Rt Hon Johnny Mercer': ['https://pbs.twimg.com/profile_images/1646621485178445831/pDovn7vE_bigger.jpg',
  'Rt Hon Johnny Mercer',
  'Rt Hon Johnny Mercer\n@JohnnyMercerUK\nFollow\nFather of three; husband of 1. Believer in change. Member of Parliament for Plymouth Moor View'],
 'fill your boots': ['https://pbs.twimg.com/profile_images/1578434262990262274/DbSW7ND1_bigger.jpg',
  'fill your boots',
  'fill your boots\n@MilitaryBanter\nFollow\nLEADERS'],
 'Navy Lookout': ['https://pbs.twimg.com/profile_images/1348937317902389251/WSHWn_iY_bigger.jpg',
  'Navy Lookout',
  'Navy Lookout\n@NavyLookout\nFollow\nIndependent Royal Navy news and analysis'],
 'Conservatives': ['https://pbs.twimg.com/profile_images/1592546189081206785/iq7d0O2x_bigger.jpg',
  'Conservatives',


In [264]:
df = pd.DataFrame.from_dict(profiles).T.astype(str).drop_duplicates().T

In [265]:
df.to_csv("All_Followers_Profiles.csv")

In [266]:
df_r = pd.read_csv("All_Followers_Profiles.csv")

In [267]:
df_r

,Unnamed: 0,James Cleverly,Rt Hon Johnny Mercer,fill your boots,Navy Lookout,Conservatives,Sir Humphrey,Think Defence,UK Defence Journal,Tom Tugendhat,...,Yazeed Althobaiti يزيد الثبيتي,Arezoo54,Vaibhav Rawat,Anil kumar saini,The Conservative Reader,Roz,Didar HS,Atul.M.Bhatt,No_alt102,No_alt121
0,0,https://pbs.twimg.com/profile_images/163920305...,https://pbs.twimg.com/profile_images/164662148...,https://pbs.twimg.com/profile_images/157843426...,https://pbs.twimg.com/profile_images/134893731...,https://pbs.twimg.com/profile_images/159254618...,https://pbs.twimg.com/profile_images/465597578...,https://pbs.twimg.com/profile_images/155693527...,https://pbs.twimg.com/profile_images/159229438...,https://pbs.twimg.com/profile_images/169736900...,...,https://pbs.twimg.com/profile_images/158170923...,https://pbs.twimg.com/profile_images/158898172...,https://pbs.twimg.com/profile_images/169239453...,https://pbs.twimg.com/profile_images/156025802...,https://pbs.twimg.com/profile_images/169979782...,https://abs.twimg.com/sticky/default_profile_i...,https://pbs.twimg.com/profile_images/167441181...,https://pbs.twimg.com/profile_images/859005012...,No_image,No_image
1,1,James Cleverly,Rt Hon Johnny Mercer,fill your boots,Navy Lookout,Conservatives,Sir Humphrey,Think Defence,UK Defence Journal,Tom Tugendhat,...,Yazeed Althobaiti يزيد الثبيتي,Arezoo54,Vaibhav Rawat,Anil kumar saini,The Conservative Reader,Roz,Didar HS,Atul.M.Bhatt,No_alt102,No_alt121
2,2,James Cleverly\n@JamesCleverly\nFollow\nForeig...,Rt Hon Johnny Mercer\n@JohnnyMercerUK\nFollow\...,fill your boots\n@MilitaryBanter\nFollow\nLEADERS,Navy Lookout\n@NavyLookout\nFollow\nIndependen...,Conservatives\n@Conservatives\nFollow\nFind ou...,Sir Humphrey\n@pinstripedline\nFollow\nAuthor ...,Think Defence\n@thinkdefence\nFollow\nPart-tim...,UK Defence Journal\n@UKDefJournal\nFollow\nInd...,Tom Tugendhat\n@TomTugendhat\nFollow\nMP for T...,...,Yazeed Althobaiti يزيد الثبيتي\n@YazeedHT\nFol...,Arezoo54\n@AyselNaz\nFollow,Vaibhav Rawat\n@VaibhavRaw21601\nFollow,Anil kumar saini\n@Anilkum06491350\nFollow,The Conservative Reader\n@TheConReaderSub\nFol...,Roz\n@Rosalinda230179\nFollow,Didar HS\n@Didarul61946396\nFollow\nচল্লিশ কোট...,Atul.M.Bhatt\n@atul_bhatt9\nFollow,NaN,NaN


In [268]:
save_2_file("overal_Followers_img_urls.txt",overal_img_urls)
save_2_file("overal_Followers_alts.txt",overal_alts)
save_2_file("Followers_full_text.txt",full_text)

In [269]:
#df_r.iloc[0,3][2:-2].replace("'","").replace(" ","").split(",")-->no good for get_element

In [270]:
df_r.iloc[0,3]

'https://pbs.twimg.com/profile_images/1578434262990262274/DbSW7ND1_bigger.jpg'

In [253]:
for i in range(1,len(df_r.columns)-1):
    j = 0
    try: 
        response = requests.get(df_r.iloc[0,i])
        file = open(f"{df_r.columns[i]}.jpeg", "wb")
        file.write(response.content)
        file.close()
#         file = open(f"Post{i}.jpeg", "wb")
#         file.write(response.content)
#         file.close()
    except Exception as e:
        print(f'An error occurred: {str(e)}')
        time.sleep(2)
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
An error occurred: [Errno 22] Invalid argument: 'The Lodge | Uppingham School.jpeg'
23
24
An error occurred: [Errno 22] Invalid argument: 'Sport | Uppingham School.jpeg'
25
26
27
28
29
30
An error occurred: [Errno 22] Invalid argument: 'Fircroft | Uppingham School.jpeg'
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
An error occurred: [Errno 2] No such file or directory: 'Forsvarsministeriet/Danish MoD.jpeg'
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
